# 3-4. モデル作成

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score

In [ ]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

## データの準備

In [ ]:
train = pd.read_csv("data/converted_train.csv")
test = pd.read_csv("data/converted_test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X_train = train.drop(["label"], axis=1)
y_train = train["label"]
X_test = test.drop(["label"], axis=1)
y_test = test["label"]

## モデルの訓練

In [ ]:
# LightGBM
param = {
    "objective": "binary",
    "boosting_type": "gbdt",
    "num_leaves": 10, 
    "n_estimators": 50,
}
lgb_model = lgb.LGBMClassifier(**param)

In [ ]:
# Random Forest
random_forest = RandomForestClassifier()

In [ ]:
# Logistic Regression
logistic_regression = LogisticRegression()

In [ ]:
# Support Vector Machine
svc = SVC()

In [ ]:
model = lgb_model
# model = random_forest
# model = logistic_regression
# model = svc

In [ ]:
model.fit(X_train, y_train)

## テスト

In [ ]:
y_pred = model.predict_proba(X_test)[:, 1]

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test, y_pred>0.5)

## アンダーサンプリング
labelの偏りが見られるため、アンダーサンプリングを試します。

In [ ]:
y_train.value_counts()

In [ ]:
sampler = RandomUnderSampler()
X_us, y_us = sampler.fit_resample(X_train, y_train)

In [ ]:
y_us.value_counts()

上ではLightGBMのscikit-learn APIを使用しましたが、ここではLightGBM独自のAPIを使用しています。

In [ ]:
train_data = lgb.Dataset(X_us, label=y_us)
# train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [ ]:
param = {
    "objective": "binary",
    "num_leaves": 10, 
    "metric": ["auc", "binary_logloss"],
}
num_boost_round = 50
train_result = {}
callbacks = [
    lgb.log_evaluation(),
    lgb.record_evaluation(train_result),
    # lgb.early_stopping(stopping_rounds=5),
]
booster = lgb.train(param, train_data, num_boost_round, valid_sets=[valid_data], valid_names=["Test"], callbacks=callbacks)

In [ ]:
y_pred = booster.predict(X_test)
accuracy_score(y_test, y_pred>0.5)

In [ ]:
# See feature importance
lgb.plot_importance(booster, figsize=(16, 6))

In [ ]:
fig = plt.figure(figsize=(16, 6))
ax1 = fig.add_subplot(111)
 
# Set labels
ax1.set_xlabel("Iteration")
ax1.set_ylabel("binary logloss")

# Plot the result
ax1.plot(train_result["Test"]["binary_logloss"], label="test loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 6))
ax1 = fig.add_subplot(111)
 
# Set labels
ax1.set_xlabel("Iteration")
ax1.set_ylabel("binary logloss")

# Plot the result
ax1.plot(train_result["Test"]["auc"], label="auc")
plt.legend()
plt.show()

次回以降も使うため、LightGBMモデルを保存します。

In [ ]:
booster.save_model("model/lightgbm_model.txt")